# Behavior Cloning Project

The goals / steps of this project are the following:
* Use the simulator to collect data of good driving behavior
* Build, a convolution neural network in Keras that predicts steering angles from images
* Train and validate the model with a training and validation set
* Test that the model successfully drives around track one without leaving the road
* Summarize the results with a written report

The simulator can be found here:

https://github.com/udacity/self-driving-car-sim 
https://github.com/endymioncheung/CarND-MacCatalinaSimulator (macOS)

### Collecting Training Data
In order to start collecting training data, you'll need to do the following:

1. Enter Training Mode in the simulator.
2. Start driving the car to get a feel for the controls.
3. When you are ready, hit the record button in the top right to start recording.
4. Continue driving for a few laps or till you feel like you have enough data.
5. Hit the record button in the top right again to stop recording.

#### Strategies for Collecting Data

    the car should stay in the center of the road as much as possible
    if the car veers off to the side, it should recover back to center
    driving counter-clockwise can help the model generalize
    flipping the images is a quick way to augment the data
    collecting data from the second track can also help generalize the model
    we want to avoid overfitting or underfitting when training the model
    knowing when to stop collecting more data

Data will be saved from the recorder as follows:

1. IMG folder - this folder contains all the frames of your driving.
2. driving_log.csv - each row in this sheet correlates your image with the steering angle, throttle, brake, and speed of your car. You'll mainly be using the steering angle.


## Training The Network

I will use Keras to train a network to do the following:

1. Take in an image from the center camera of the car. This is the input to your neural network.
2. Output a new steering angle for the car.

The following network will be used to verify that everything is working properly and will be a flattened image connected to a single output node. 
This single output node will predict my steering angle, which makes this a regression network and no activation function will be applied. 
This will try to minimize the error that the network predicts and the ground truth steering measurement

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import csv
import cv2
import numpy as np
import matplotlib.pyplot as plt

lines = []
# Loading in the data from the csv file
with open('data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)

# Extracting the image and measurements 
images = []
measurements = []
for line in lines[1:]:
    source_path = line[0]
    filename = source_path.split('/')[-1]
    current_path = 'data/IMG/' + filename
    image = cv2.imread(current_path)
    images.append(image)
    measurement = float(line[3])
    measurements.append(measurement)
    
X_train = np.array(images)
y_train = np.array(measurements)

from keras.models import Sequential
from keras.layers import Flatten, Dense
model = Sequential()
model.add(Flatten(input_shape = (160,320,3)))
model.add(Dense(1))

# Using mean square error for regression network
model.compile(loss = 'mse', optimizer = 'adam')
model.fit(X_train, y_train, validation_split=0.2, shuffle = True, nb_epoch=7)

model.save('model.h5')

NOTE: cv2.imread will get images in BGR format, while drive.py uses RGB. One way to keep the same image formatting is to do "image = ndimage.imread(current_path)" with "from scipy import ndimage" instead.



## Training The Network Continued 

I will use Keras to continue training the network by adding in the following:

### Lambda Layers
In Keras, lambda layers can be used to create arbitrary functions that operate on each image as it passes through the layer.

In this project, a lambda layer is a convenient way to parallelize image normalization. The lambda layer will also ensure that the model will normalize input images when making predictions in drive.py.

### Flipping Images And Steering Measurements
A effective technique for helping with the left turn bias involves flipping images and taking the opposite sign of the steering measurement. 

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import csv
import cv2
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Lambda

lines = []
# Loading in the data from the csv file
with open('data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)

# Extracting the image and measurements 
images = []
measurements = []
for line in lines[1:]:
    source_path = line[0]
    filename = source_path.split('/')[-1]
    current_path = 'data/IMG/' + filename
    image = cv2.imread(current_path)
    images.append(image)
    measurement = float(line[3])
    measurements.append(measurement)
    
augmented_images, augmented_measurements = [], []
for image, measurement in zip(images, measurements):
    augmented_images.append(image)
    augmented_measurements.append(measurement)
    augmented_images.append(cv2.flip(image,1))
    augmented_measurements.append(measurement*-1.0)
    
X_train = np.array(augmented_images)
y_train = np.array(augmented_measurements)

from keras.models import Sequential
from keras.layers import Flatten, Dense
model = Sequential()
# set up lambda layer for normalization
model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))
model.add(Flatten())
model.add(Dense(1))

# Using mean square error for regression network
model.compile(loss = 'mse', optimizer = 'adam')
model.fit(X_train, y_train, validation_split=0.2, shuffle = True, nb_epoch=1)

model.save('model.h5')

## Multiple Cameras

In a real car, we’ll have multiple cameras on the vehicle, and we’ll map recovery paths from each camera. For example, if you train the model to associate a given image from the center camera with a left turn, then you could also train the model to associate the corresponding image from the left camera with a somewhat softer left turn. And you could train the model to associate the corresponding image from the right camera with an even harder left turn.

In that way, you can simulate your vehicle being in different positions, somewhat further off the center line. To read more about this approach, see this paper by NVIDIA: http://images.nvidia.com/content/tegra/automotive/images/2016/solutions/pdf/end-to-end-dl-using-px.pdf

Explanation of How Multiple Cameras Work

From the perspective of the left camera, the steering angle would be less than the steering angle from the center camera. From the right camera's perspective, the steering angle would be larger than the angle from the center camera. The next section will discuss how this can be implemented in your project although there is no requirement to use the left and right camera images.

Multiple Cameras in This Project

For this project, recording recoveries from the sides of the road back to center is effective. But it is also possible to use all three camera images to train the model. When recording, the simulator will simultaneously save an image for the left, center and right cameras. Each row of the csv log file, driving_log.csv, contains the file path for each camera as well as information about the steering measurement, throttle, brake and speed of the vehicle.

During training, I want to feed the left and right camera images to your model as if they were coming from the center camera. This way, you can teach your model how to steer if the car drifts off to the left or the right.

Figuring out how much to add or subtract from the center angle will involve some experimentation.

During prediction (i.e. "autonomous mode"), you only need to predict with the center camera image.

It is not necessary to use the left and right images to derive a successful model. Recording recovery driving from the sides of the road is also effective.


In [ ]:
import warnings
warnings.filterwarnings("ignore")

import csv
import cv2
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Lambda

lines = []
# Loading in the data from the csv file
with open('data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)

# create adjusted steering measurements for the side camera images
correction = 0.2 # this is a parameter to tune

# Extracting the image and measurements 
images = []
measurements = []
for line in lines[1:]:
    for i in range(3):
        source_path = line[i]
        filename = source_path.split('/')[-1]
        current_path = 'data/IMG/' + filename
        image = cv2.imread(current_path)
        images.append(image)
        if i == 0: # Center
            measurement = float(line[3])
        elif i == 1: # Left
            measurement = float(line[3]) + correction
        elif i == 2: # Right
            measurement = float(line[3]) - correction        
        measurements.append(measurement)
    
augmented_images, augmented_measurements = [], []
for image, measurement in zip(images, measurements):
    augmented_images.append(image)
    augmented_measurements.append(measurement)
    augmented_images.append(cv2.flip(image,1))
    augmented_measurements.append(measurement*-1.0)
    
X_train = np.array(augmented_images)
y_train = np.array(augmented_measurements)

from keras.models import Sequential
from keras.layers import Flatten, Dense
model = Sequential()
# set up lambda layer for normalization
model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))
model.add(Flatten())
model.add(Dense(1))

# Using mean square error for regression network
model.compile(loss = 'mse', optimizer = 'adam')
model.fit(X_train, y_train, validation_split=0.2, shuffle = True, nb_epoch=1)

model.save('model.h5')

## Cropping2D Layer

Keras provides the Cropping2D layer for image cropping within the model. This is relatively fast, because the model is parallelized on the GPU, so many images are cropped simultaneously.

By contrast, image cropping outside the model on the CPU is relatively slow.

Also, by adding the cropping layer, the model will automatically crop the input images when making predictions in drive.py.

The Cropping2D layer might be useful for choosing an area of interest that excludes the sky and/or the hood of the car.

Cropping Layer Code Example
    
    from keras.models import Sequential, Model
    from keras.layers import Cropping2D
    import cv2

    # set up cropping2D layer
    model = Sequential()
    model.add(Cropping2D(cropping=((50,20), (0,0)), input_shape=(160,320,3)))


The example above crops:

* 50 rows pixels from the top of the image
* 20 rows pixels from the bottom of the image
* 0 columns of pixels from the left of the image
* 0 columns of pixels from the right of the image

## Network Architecture Updates

I am going to implement the NVIDIA Self Driving Car network published here: https://devblogs.nvidia.com/deep-learning-self-driving-cars/

The first layer of the network performs image normalization. The normalizer is hard-coded and is not adjusted in the learning process. Performing normalization in the network allows the normalization scheme to be altered with the network architecture, and to be accelerated via GPU processing.

The convolutional layers are designed to perform feature extraction, and are chosen empirically through a series of experiments that vary layer configurations. We then use strided convolutions in the first three convolutional layers with a 2×2 stride and a 5×5 kernel, and a non-strided convolution with a 3×3 kernel size in the final two convolutional layers.

We follow the five convolutional layers with three fully connected layers, leading to a final output control value which is the inverse-turning-radius. The fully connected layers are designed to function as a controller for steering, but we noted that by training the system end-to-end, it is not possible to make a clean break between which parts of the network function primarily as feature extractor, and which serve as controller.




In [ ]:
import warnings
warnings.filterwarnings("ignore")

import csv
import cv2
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Lambda, Cropping2D
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.models import Sequential
from keras.layers import Flatten, Dense

lines = []
# Loading in the data from the csv file
with open('data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)

# create adjusted steering measurements for the side camera images
correction = 0.2 # this is a parameter to tune

# Extracting the image and measurements 
images = []
measurements = []
for line in lines[1:]:
    for i in range(3):
        source_path = line[i]
        filename = source_path.split('/')[-1]
        current_path = 'data/IMG/' + filename
        imageBGR = cv2.imread(current_path)
        # Images in drive.py are read in as RGB
        image = cv2.cvtColor(imageBGR, cv2.COLOR_BGR2RGB)
        images.append(image)
        if i == 0: # Center
            measurement = float(line[3])
        elif i == 1: # Left
            measurement = float(line[3]) + correction
        elif i == 2: # Right
            measurement = float(line[3]) - correction        
        measurements.append(measurement)
    
augmented_images, augmented_measurements = [], []
for image, measurement in zip(images, measurements):
    augmented_images.append(image)
    augmented_measurements.append(measurement)
    augmented_images.append(cv2.flip(image,1))
    augmented_measurements.append(measurement*-1.0)
    
X_train = np.array(augmented_images)
y_train = np.array(augmented_measurements)

model = Sequential()
# set up lambda layer for normalization
model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))
# cropping 70 pixels from top of image (trees) and 25 pixels from bottom of image (hood of car)
model.add(Cropping2D(cropping=((70,25),(0,0))))

# NVIDIA architecture and including a dropout layer for redundancy
model.add(Conv2D(24,5,5, subsample = (2,2), activation = "relu"))
model.add(Conv2D(36,5,5, subsample = (2,2), activation = "relu"))
model.add(Conv2D(48,5,5, subsample = (2,2), activation = "relu"))
model.add(Conv2D(64,3,3, activation = "relu"))
model.add(Conv2D(64,3,3, activation = "relu"))
model.add(Flatten())
model.add(Dropout(0.5))   
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))

# Using mean square error for regression network
model.compile(loss = 'mse', optimizer = 'adam')
model.fit(X_train, y_train, validation_split=0.2, shuffle = True, nb_epoch=5)

# Saving to file for use in drive.py
model.save('model.h5')

## Generators 

The images captured in the car simulator are much larger than the images encountered in the Traffic Sign Classifier Project, a size of 160 x 320 x 3 compared to 32 x 32 x 3. Storing 10,000 traffic sign images would take about 30 MB but storing 10,000 simulator images would take over 1.5 GB. That's a lot of memory! Not to mention that preprocessing data can change data types from an int to a float, which can increase the size of the data by a factor of 4.

Generators can be a great way to work with large amounts of data. Instead of storing the preprocessed data in memory all at once, using a generator you can pull pieces of the data and process them on the fly only when you need them, which is much more memory-efficient.

A generator is like a coroutine, a process that can run separately from another main routine, which makes it a useful Python function. Instead of using return, the generator uses yield, which still returns the desired output values but saves the current values of all the generator's variables. When the generator is called a second time it re-starts right after the yield statement, with all its variables set to the same values as before.



In [19]:
import os
import csv
import warnings
warnings.filterwarnings("ignore")
import cv2
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Lambda, Cropping2D
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers.convolutional import Conv2D
import sklearn
from sklearn.utils import shuffle

samples = []
# Loading in the data from the csv file
with open('data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        samples.append(line)

# create adjusted steering measurements for the side camera images
correction = 0.2 # this is a parameter to tune

from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(samples, test_size=0.2)

def generator(samples, batch_size=32):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]
            images = []
            measurements = []
            for batch_sample in batch_samples:
                for i in range(3):
                    source_path = batch_sample[i]
                    filename = source_path.split('/')[-1]
                    current_path = 'data/IMG/' + filename
                    imageBGR = cv2.imread(current_path)
                    # Images in drive.py are read in as RGB
                    image = cv2.cvtColor(imageBGR, cv2.COLOR_BGR2RGB)
                    images.append(image)
                    if i == 0: # Center
                        measurement = float(line[3])
                    elif i == 1: # Left
                        measurement = float(line[3]) + correction
                    elif i == 2: # Right
                        measurement = float(line[3]) - correction        
                    measurements.append(measurement)

            # trim image to only see section with road
            X_train = np.array(images)
            y_train = np.array(measurements)
            yield sklearn.utils.shuffle(X_train, y_train)

# Set our batch size
batch_size=32

# compile and train the model using the generator function
train_generator = generator(train_samples, batch_size=batch_size)
validation_generator = generator(validation_samples, batch_size=batch_size)

model = Sequential()
# set up lambda layer for normalization
model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))
# cropping 70 pixels from top of image (trees) and 25 pixels from bottom of image (hood of car)
model.add(Cropping2D(cropping=((70,25),(0,0))))

# NVIDIA architecture and including a dropout layer for redundancy
model.add(Conv2D(24,5,5, subsample = (2,2), activation = "relu"))
model.add(Conv2D(36,5,5, subsample = (2,2), activation = "relu"))
model.add(Conv2D(48,5,5, subsample = (2,2), activation = "relu"))
model.add(Conv2D(64,3,3, activation = "relu"))
model.add(Conv2D(64,3,3, activation = "relu"))
model.add(Flatten())
model.add(Dropout(0.5))   
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
history_object = model.fit_generator(train_generator, steps_per_epoch=np.ceil(len(train_samples)/batch_size), 
                    validation_data=validation_generator, 
                    validation_steps=np.ceil(len(validation_samples)/batch_size), 
                    epochs=5, verbose=1)

### print the keys contained in the history object
print(history_object.history.keys())

### plot the training and validation loss for each epoch
plt.plot(history_object.history['loss'])
plt.plot(history_object.history['val_loss'])
plt.title('model mean squared error loss')
plt.ylabel('mean squared error loss')
plt.xlabel('epoch')
plt.legend(['training set', 'validation set'], loc='upper right')
plt.show()

# Save the model
model.save('model.h5')

Epoch 1/5
 98/201 [=============>................] - ETA: 34s - loss: 0.0145

error: OpenCV(4.2.0) ../modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


## Visualizin Loss

Outputting Training and Validation Loss Metrics
In Keras, the model.fit() and model.fit_generator() methods have a verbose parameter that tells Keras to output loss metrics as the model trains. The verbose parameter can optionally be set to verbose = 1 or verbose = 2.

Setting model.fit(verbose = 1) will

output a progress bar in the terminal as the model trains.
output the loss metric on the training set as the model trains.
output the loss on the training and validation sets after each epoch.
With model.fit(verbose = 2), Keras will only output the loss on the training set and validation set after each epoch.



In [ ]:
from keras.models import Model
import matplotlib.pyplot as plt

history_object = model.fit_generator(train_generator, samples_per_epoch =
    len(train_samples), validation_data = 
    validation_generator,
    nb_val_samples = len(validation_samples), 
    nb_epoch=3, verbose=1)

### print the keys contained in the history object
print(history_object.history.keys())

### plot the training and validation loss for each epoch
plt.plot(history_object.history['loss'])
plt.plot(history_object.history['val_loss'])
plt.title('model mean squared error loss')
plt.ylabel('mean squared error loss')
plt.xlabel('epoch')
plt.legend(['training set', 'validation set'], loc='upper right')
plt.show()

# Save the model
model.save('model.h5')

## Validating The Network

In order to validate your network, you'll want to compare model performance on the training set and a validation set. The validation set should contain image and steering data that was not used for training. A rule of thumb could be to use 80% of your data for training and 20% for validation or 70% and 30%. Be sure to randomly shuffle the data before splitting into training and validation sets.

If model predictions are poor on both the training and validation set (for example, mean squared error is high on both), then this is evidence of underfitting. Possible solutions could be to

    increase the number of epochs
    add more convolutions to the network.
    
When the model predicts well on the training set but poorly on the validation set (for example, low mean squared error for training set, high mean squared error for validation set), this is evidence of overfitting. If the model is overfitting, a few ideas could be to

    use dropout or pooling layers
    use fewer convolution or fewer fully connected layers
    collect more data or further augment the data set
    
Ideally, the model will make good predictions on both the training and validation sets. The implication is that when the network sees an image, it can successfully predict what angle was being driven at that moment.

## Testing The Network

Once you're satisfied that the model is making good predictions on the training and validation sets, you can test your model by launching the simulator and entering autonomous mode.

The car will just sit there until your Python server connects to it and provides it steering angles. Here’s how you start your Python server:

    python drive.py model.h5

Once the model is up and running in drive.py, you should see the car move around (and hopefully not off) the track! If your model has low mean squared error on the training and validation sets but is driving off the track, this could be because of the data collection process. It's important to feed the network examples of good driving behavior so that the vehicle stays in the center and recovers when getting too close to the sides of the road.

In [17]:
!python drive.py model.h5 run2

Using TensorFlow backend.
/opt/anaconda3/envs/carnd-term1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda3/envs/carnd-term1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda3/envs/carnd-term1/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda3/envs/carnd-ter

0.005590744316577911 6.974694600000006
0.005590744316577911 6.993101600000006
0.027475766837596893 6.986763400000006
0.033414147794246674 6.988181600000006
0.012549229897558689 6.982258600000007
0.012549229897558689 6.999395600000007
0.035500433295965195 6.9990702000000065
0.07779770344495773 6.993424800000006
0.04022946581244469 6.991521400000006
0.04022946581244469 7.007518000000006
0.06290223449468613 7.001941600000006
0.09686023741960526 6.998767600000006
0.09839380532503128 6.993104400000006
0.09839380532503128 7.007901200000006
0.09216853231191635 7.003460800000007
0.09458433091640472 6.997613000000007
0.09458433091640472 7.011635200000008
0.1017889752984047 7.006308000000008
0.11212877184152603 6.999948600000008
0.10488542169332504 6.993686600000007
0.10488542169332504 7.006554600000008
0.07477135956287384 6.999634600000007
0.10016938298940659 6.992622400000007
0.10016938298940659 7.004720200000007
0.07111763209104538 6.977650000000007
0.03821811079978943 6.969375400000007
0.038

0.017672428861260414 6.534204400000003
0.009246542118489742 6.533820600000002
0.015258628875017166 6.533324400000002
0.015258628875017166 6.5329482000000025
-0.015258070081472397 6.532531200000002
-0.0987250804901123 6.532144000000002
-0.0987250804901123 6.531766800000002
-0.07752494513988495 6.531399800000002
-0.11086287349462509 6.5310228000000015
-0.11086287349462509 6.530645800000001
-0.062287475913763046 6.530523800000001
-0.05654587969183922 6.530141600000002
-0.05654587969183922 6.529769400000002
-0.10458695143461227 6.529468600000002
-0.06769414246082306 6.528873400000001
-0.06769414246082306 6.528498200000001
-0.08635812252759933 6.5280312
-0.05773618817329407 6.527807200000001
-0.05773618817329407 6.527433200000002
-0.11226307600736618 6.526977600000001
-0.06074761599302292 6.5266122000000015
-0.06074761599302292 6.526236800000001
-0.044426530599594116 6.5257900000000015
-0.07931016385555267 6.525535600000001
-0.07931016385555267 6.525161200000002
-0.08519530296325684 6.52469

-0.025839755311608315 6.4524236
-0.025839755311608315 6.4520464
-0.006716864183545113 6.4516794
0.04002295061945915 6.4513126000000005
0.011832792311906815 6.4509154
0.011832792311906815 6.4505382
-0.03780333697795868 6.4501610000000005
0.0060050031170248985 6.449855200000001
-0.03243358060717583 6.449469200000001
-0.03243358060717583 6.449093200000001
0.00828476157039404 6.448676400000001
-0.014098824001848698 6.448289399999999
-0.014694726094603539 6.4479226
-0.014694726094603539 6.4475458
0.0035363140050321817 6.4471587999999995
-0.06731230020523071 6.446781799999999
0.006414378993213177 6.4463946
0.006414378993213177 6.4460174
-0.0046857791021466255 6.4456402
-0.010783582925796509 6.4453854
-0.010783582925796509 6.4450106
-0.010291418060660362 6.444523599999999
-0.026483990252017975 6.444146599999999
-0.026483990252017975 6.443769599999999
-0.004256190732121468 6.443382399999999
-0.013472137041389942 6.443005199999999
-0.013472137041389942 6.442627999999999
-0.02118515595793724 6.4

-0.009244604036211967 6.371724800000002
-0.009244604036211967 6.371127200000002
-0.0696694403886795 6.370780200000001
-0.04976461082696915 6.370382800000002
-0.04976461082696915 6.370005400000001
-0.011883405968546867 6.369668800000001
-0.03671666979789734 6.369292200000001
-0.005438640713691711 6.368915600000002
-0.005438640713691711 6.368539000000002
-0.005438640713691711 6.3681420000000015
0.0017643258906900883 6.367754800000002
0.0034909553360193968 6.367367400000002
0.0034909553360193968 6.366990000000002
0.01398657076060772 6.366622800000002
0.022180918604135513 6.366255800000001
0.004706885665655136 6.365878800000001
0.004706885665655136 6.3655018
0.019861264154314995 6.365135
0.047594718635082245 6.364768400000001
0.047594718635082245 6.364391800000001
0.07954718172550201 6.3640050000000015
0.01845364272594452 6.363628200000002
0.01845364272594452 6.363251400000001
0.00814132858067751 6.362915400000001
-0.005273830145597458 6.3626516
-0.005273830145597458 6.3622778
0.0482877753

...

0.030562154948711395 35.69445919999997
0.030562154948711395 35.75445439999997
0.031643617898225784 35.81339899999997
0.026326820254325867 35.873322599999966
0.026326820254325867 35.93329619999996
0.024470090866088867 35.994350999999966
0.025868181139230728 36.05399899999996
0.025868181139230728 36.11398699999996
0.024394340813159943 36.17327119999996
0.029010429978370667 36.23324539999995
0.029010429978370667 36.29285239999995
0.029010429978370667 36.352819399999944
0.0262986458837986 36.41220499999994
0.020790942013263702 36.473262199999944
0.020790942013263702 36.53323939999995
0.025322746485471725 36.59297179999995
0.02627900429069996 36.65344399999994
0.02627900429069996 36.713426199999944
0.02460448071360588 36.77379599999995
0.021235983818769455 36.831694799999944
0.021235983818769455 36.89164359999994
0.029060181230306625 36.952999999999946
0.023491591215133667 37.012680599999946
0.023491591215133667 37.072651199999946
0.02613002434372902 37.133029799999946
0.022189170122146606 

0.014568417333066463 48.34708199999997
0.014568417333066463 48.40706079999997
0.006551627069711685 48.467345599999966
0.010442203842103481 48.52787099999996
0.011037726886570454 48.58726459999997
0.011037726886570454 48.64724819999997
0.006864693947136402 48.70735419999996
0.010438489727675915 48.767880799999965
0.011581804603338242 48.826969599999956
0.011051538400352001 48.88644859999995
0.011051538400352001 48.946417599999954
0.010805990546941757 49.00742699999996
0.014995570294559002 49.06740619999996
0.014995570294559002 49.12739539999996
0.01318526454269886 49.18723159999996
0.011789276264607906 49.24775839999996
0.012654266320168972 49.30693919999996
0.007948171347379684 49.36639979999996
0.007948171347379684 49.42637039999996
0.0014472824987024069 49.486004399999956
0.014021823182702065 49.54596839999996
0.014021823182702065 49.60679939999996
0.014021823182702065 49.666780399999965
0.007951007224619389 49.726455399999956
0.006074385717511177 49.78643039999996
0.0060743857175111

-0.16378940641880035 57.919614799999984
-0.16378940641880035 57.95279019999998
-0.1722990870475769 57.971277599999986
-0.1579228937625885 57.97269799999999
-0.1579228937625885 58.00496839999999
-0.14264458417892456 58.03723879999999
-0.14264458417892456 58.061012599999984
-0.17075581848621368 58.057069599999984
-0.17075581848621368 58.08846659999998
-0.17002572119235992 58.11288679999998
-0.18666309118270874 58.10674359999997
-0.1833450049161911 58.13186439999998
-0.1833450049161911 58.16228519999998
-0.1696733981370926 58.19270599999997
-0.1696733981370926 58.18468299999998
-0.1705988645553589 58.21018839999998
-0.1705988645553589 58.23977379999999
-0.16872920095920563 58.230099399999986
-0.15182174742221832 58.255242999999986
-0.1433584839105606 58.24464519999998
-0.1433584839105606 58.27261739999998
-0.13473838567733765 58.300589599999974
-0.13473838567733765 58.32484899999998
-0.14267562329769135 58.31375379999998
-0.14267562329769135 58.340888599999985
-0.1492169350385666 58.36388

0.06151244044303894 58.44701219999999
0.043669916689395905 58.44663479999999
0.043669916689395905 58.44625739999999
0.043669916689395905 58.44590039999999
0.026278287172317505 58.44552339999999
0.05017166957259178 58.44514639999999
0.05017166957259178 58.44475919999999
0.07714761048555374 58.444361599999986
0.07714761048555374 58.443983999999986
0.11154435575008392 58.443606399999986
0.11154435575008392 58.44325939999999
0.1517922580242157 58.44288239999999
0.1517922580242157 58.44254619999999
0.1517922580242157 58.44216999999999
0.14745326340198517 58.44179379999999
0.14745326340198517 58.432186599999994
0.12801411747932434 58.488800399999995
0.11052969098091125 58.4893642
0.11052969098091125 58.4208332
0.12325578927993774 58.420042200000005
0.12325578927993774 58.419251200000005
0.12325578927993774 58.50355880000001
0.129765123128891 58.451274000000005
0.11181215196847916 58.451109200000005
0.11181215196847916 58.44947560000001
0.09584231674671173 58.449282000000004
0.095842316746711

^C
wsgi exiting
(24141) wsgi exited, is_accepting=True


The fourth argument, run1, is the directory in which to save the images seen by the agent. If the directory already exists, it'll be overwritten.

The image file name is a timestamp of when the image was seen.. This information is used by video.py to create a chronological video of the agent driving.

Using video.py

    python video.py run1
    
Creates a video based on images found in the run1 directory. The name of the video will be the name of the directory followed by '.mp4', so, in this case the video will be run1.mp4.

Optionally, one can specify the FPS (frames per second) of the video:

    python video.py run1 --fps 48
    
The video will run at 48 FPS. The default FPS is 60.



In [ ]:
!python video.py run2